#### 29.10.25, &copy; [Evhenii Kostin](https://github.com/DE123MasterProgram2025autumn/DE_Kostin), 2025

# Лабораторна робота №3:Робота зі структурованими даними, перетворення форматів та віконні операції

__Мета:__ _освоїти сучасні методи обробки неструктурованих та напівструктурованих даних (JSON), навчитися перетворювати формати таблиць (широкий ↔︎ довгий) за допомогою tidyr, а також застосовувати віконні функції для аналізу даних у контексті груп (наприклад, часових рядів, відділів, регіонів). Ці навички є ключовими для підготовки даних у складних ETL-процесах._

### Варіант 2
#### Тема: Аналіз даних про веб-трафік та маркетингові кампанії

In [1]:
# Виконав(ла): Костін Євгеній | Група: КІ-24-1м | Викладач: Сидоренко В.М.
# Лабораторна робота №3: Єдиний скрипт (Генерація + Рішення)

import pandas as pd
import json
import os
import numpy as np

# --- ЕТАП 0: ГЕНЕРАЦІЯ ДАНИХ (Setup) ---
# Ця частина створює файли, які будуть потрібні для Етапів 1-4

print("--- ЕТАП 0: СТВОРЕННЯ ВИХІДНИХ ФАЙЛІВ ---")

# --- 1. НАЛАШТУВАННЯ ТА СТВОРЕННЯ ПАПКИ ---
data_dir = "data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"Створено папку: {data_dir}")
else:
    print(f"Папка {data_dir} вже існує.")

# --- 2. ГЕНЕРАЦІЯ ДАНИХ ---

# 2.1. traffic.json
traffic_data = [
  {
    "date": "2024-01-05", "page_views": 1250, "unique_users": 840, "bounce_rate": 0.42,
    "campaigns": [
      {"name": "google_ads", "channel": "paid_search", "cost": 120},
      {"name": "email_jan", "channel": "email", "cost": 50}
    ]
  },
  {
    "date": "2024-01-15", "page_views": 1300, "unique_users": 900, "bounce_rate": 0.40,
    "campaigns": [
      {"name": "google_ads", "channel": "paid_search", "cost": 130},
      {"name": "facebook_jan", "channel": "social", "cost": 0} # Вартість 0
    ]
  },
  {
    "date": "2024-02-05", "page_views": 1100, "unique_users": 750, "bounce_rate": 0.45, # Bounce rate зріс
    "campaigns": [
      {"name": "google_ads", "channel": "paid_search", "cost": 110},
      {"name": "email_feb", "channel": "email", "cost": 40}
    ]
  },
  {
    "date": "2024-02-15", "page_views": 1150, "unique_users": 780, "bounce_rate": 0.44,
    "campaigns": [
      {"name": "google_ads", "channel": "paid_search", "cost": 115}
    ]
  },
    {
    "date": "2024-03-05", "page_views": 1400, "unique_users": 950, "bounce_rate": 0.38, # Bounce rate впав
    "campaigns": [
      {"name": "google_ads", "channel": "paid_search", "cost": 140},
      {"name": "email_mar", "channel": "email", "cost": 55}
    ]
  }
]

# Запис JSON-файлу
json_path = os.path.join(data_dir, 'traffic.json')
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(traffic_data, f, ensure_ascii=False, indent=2)
print(f"Файл 'traffic.json' створено.")

# 2.2. monthly_reports.xlsx
df_jan = pd.DataFrame({'page': ['/home', '/products', '/contact'], 'sessions': [15000, 8000, 1500], 'avg_duration_sec': [120, 180, 90]})
df_feb = pd.DataFrame({'page': ['/home', '/products', '/contact', '/blog'], 'sessions': [13000, 7500, 1400, 1000], 'avg_duration_sec': [110, 175, 95, 200]})
df_mar = pd.DataFrame({'page': ['/home', '/products', '/contact', '/blog'], 'sessions': [16000, 8500, 1600, 1200], 'avg_duration_sec': [125, 185, 90, 210]})

excel_path = os.path.join(data_dir, 'monthly_reports.xlsx')
try:
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        df_jan.to_excel(writer, sheet_name='jan', index=False)
        df_feb.to_excel(writer, sheet_name='feb', index=False)
        df_mar.to_excel(writer, sheet_name='mar', index=False)
    print(f"Файл 'monthly_reports.xlsx' створено.")
except Exception as e:
    print(f"Помилка запису Excel: {e}. (Можливо, потрібно: !pip install openpyxl)")

# 2.3. goals.csv
goals_df = pd.DataFrame({'metric': ['total_users', 'total_sessions'], 'target_value': [2500, 50000]})
csv_path = os.path.join(data_dir, 'goals.csv')
goals_df.to_csv(csv_path, index=False)
print(f"Файл 'goals.csv' створено.")
print("--- ЕТАП 0 ЗАВЕРШЕНО. ПОЧАТОК АНАЛІЗУ (ЕТАПИ 1-4) ---\n")


# --- ЕТАП 1: РОБОТА З JSON ---
print("--- ЕТАП 1: РОБОТА З JSON ---")

# 1.1. Завантажуємо JSON (шлях з Етапу 0)
with open(json_path, 'r', encoding='utf-8') as f:
    json_data = json.load(f)

# 1.2. Перетворюємо у DataFrame та "розгортаємо"
daily_campaign_metrics = pd.json_normalize(
    json_data,
    record_path=['campaigns'],
    meta=['date', 'page_views', 'unique_users', 'bounce_rate']
)

# 1.3. Залишаємо лише записи, де вартість кампанії > 0
daily_campaign_metrics = daily_campaign_metrics[daily_campaign_metrics['cost'] > 0].copy()

print("Результат Етапу 1 (daily_campaign_metrics):")
print(daily_campaign_metrics)


# --- ЕТАП 2: ПЕРЕТВОРЕННЯ ФОРМАТІВ ---
print("\n--- ЕТАП 2: ПЕРЕТВОРЕННЯ ФОРМАТІВ ---")

# 2.1. Завантажуємо всі аркуші з monthly_reports.xlsx (шлях з Етапу 0)
all_sheets = pd.read_excel(excel_path, sheet_name=None)

# 2.2. Об'єднуємо аркуші у єдину таблицю
monthly_sheets = []
for month_name, sheet_df in all_sheets.items():
    sheet_df['month_name'] = month_name
    monthly_sheets.append(sheet_df)
monthly_traffic = pd.concat(monthly_sheets, ignore_index=True)

print("Результат 2.2 (monthly_traffic):")
print(monthly_traffic.head())

# 2.3. Завантажуємо goals.csv (шлях з Етапу 0)
goals = pd.read_csv(csv_path)
print("\nРезультат 2.3 (goals):")
print(goals)


# --- ЕТАП 3: ОБ’ЄДНАННЯ ТА ВІКОННІ ОПЕРАЦІЇ ---
print("\n--- ЕТАП 3: ВІКОННІ ОПЕРАЦІЇ ---")

# 3.1. Перетворюємо 'date' у 'month' для агрегації
daily_campaign_metrics['month'] = pd.to_datetime(daily_campaign_metrics['date']).dt.strftime('%Y-%m')

# 3.2. Агрегуємо щоденні дані по кампаніях до рівня місяця
monthly_summary = daily_campaign_metrics.groupby('month').agg(
    page_views=('page_views', 'mean'),
    unique_users=('unique_users', 'mean'),
    bounce_rate=('bounce_rate', 'mean'),
    total_cost=('cost', 'sum')
).reset_index()

# 3.3. Розрахунок конверсії
monthly_summary['conversion'] = monthly_summary['unique_users'] / monthly_summary['page_views']

# 3.4. Розрахунок ROI
monthly_summary['ROI'] = (monthly_summary['unique_users'] * 0.1 - monthly_summary['total_cost']) / monthly_summary['total_cost']

# --- 3.5. Віконні функції ---
monthly_summary = monthly_summary.sort_values(by='month')

# Ранжування: ROI
monthly_summary['roi_rank'] = monthly_summary['ROI'].rank(ascending=False, method='min')

# Зсув: зміна bounce rate
monthly_summary['bounce_change'] = monthly_summary['bounce_rate'].diff()

# Накопичення: унікальні користувачі
monthly_summary['cum_users'] = monthly_summary['unique_users'].cumsum()

print("Результат Етапу 3 (monthly_summary з віконними функціями):")
print(monthly_summary)


# --- ЕТАП 4: АНАЛІЗ ---
print("\n--- ЕТАП 4: АНАЛІЗ ---")

# 4.1. Перевірка досягнення цілей
target_users_value = goals[goals['metric'] == 'total_users']['target_value'].iloc[0]
final_cum_users = monthly_summary['cum_users'].iloc[-1] # Останнє значення

print(f"Ціль по 'total_users': {target_users_value}")
print(f"Накопичено користувачів (сума середньоденних): {final_cum_users:.2f}")
if final_cum_users >= target_users_value:
    print(">>> Ціль досягнуто!")
else:
    print(">>> Ціль не досягнуто.")

# 4.2. Аналог "semi_join": місяці, коли *всі* кампанії мали ROI > 0
print("\n4.2. Аналіз 'Semi-Join' (місяці, де ROI > 0 для *всіх* кампаній):")

# Розраховуємо ROI для *кожної окремої* кампанії (з Етапу 1)
daily_campaign_metrics['ROI'] = (daily_campaign_metrics['unique_users'] * 0.1 - daily_campaign_metrics['cost']) / daily_campaign_metrics['cost']
grouped_by_month = daily_campaign_metrics.groupby('month')

# .filter() для перевірки, чи *всі* (all) ROI в групі > 0
good_months_df = grouped_by_month.filter(lambda x: (x['ROI'] > 0).all())

if good_months_df.empty:
    print("Не знайдено місяців, де *всі* кампанії мали ROI > 0.")
else:
    print("Місяці, де всі кампанії мали ROI > 0:")
    print(good_months_df['month'].unique())


# 4.3. Аналог "anti_join": місяці, коли були витрати, але bounce rate зріс
print("\n4.3. Аналіз 'Anti-Join' (витрати були, bounce rate зріс):")
# Використовуємо 'monthly_summary' з Етапу 3
bad_months_df = monthly_summary[
    (monthly_summary['total_cost'] > 0) & 
    (monthly_summary['bounce_change'] > 0)
]

if bad_months_df.empty:
    print("Не знайдено місяців, де одночасно були витрати і зріс bounce rate.")
else:
    print("Місяці з витратами та зростанням bounce rate:")
    print(bad_months_df[['month', 'total_cost', 'bounce_change']])

print("\n--- АНАЛІЗ ЗАВЕРШЕНО ---")

--- ЕТАП 0: СТВОРЕННЯ ВИХІДНИХ ФАЙЛІВ ---
Папка data вже існує.
Файл 'traffic.json' створено.
Файл 'monthly_reports.xlsx' створено.
Файл 'goals.csv' створено.
--- ЕТАП 0 ЗАВЕРШЕНО. ПОЧАТОК АНАЛІЗУ (ЕТАПИ 1-4) ---

--- ЕТАП 1: РОБОТА З JSON ---
Результат Етапу 1 (daily_campaign_metrics):
         name      channel  cost        date page_views unique_users  \
0  google_ads  paid_search   120  2024-01-05       1250          840   
1   email_jan        email    50  2024-01-05       1250          840   
2  google_ads  paid_search   130  2024-01-15       1300          900   
4  google_ads  paid_search   110  2024-02-05       1100          750   
5   email_feb        email    40  2024-02-05       1100          750   
6  google_ads  paid_search   115  2024-02-15       1150          780   
7  google_ads  paid_search   140  2024-03-05       1400          950   
8   email_mar        email    55  2024-03-05       1400          950   

  bounce_rate  
0        0.42  
1        0.42  
2         0.4  

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6a083947-94ba-475d-8730-27cff0574f54' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>